In [9]:
import math
import random
import pygame
from pygame.locals import *
from OpenGL.GL import *
from OpenGL.GLU import *
import sys

# Size of canvas
width = 1000
height = 750
boidneighbor = 5
numBoids = 100
visualRange = 75
wind_interval = 300  # Interval in frames for wind gusts
wind_duration = 10  # Duration of each wind gust in frames
wind_direction = random.uniform(0, 2 * math.pi)  # Initialize before function calling

boids = []
emo_boid_indices = set()
frame_count = 0

def initBoids():
    global boids, emo_boid_indices
    boids = []
    for i in range(numBoids):
        boids.append({
            'x': random.random() * width,
            'y': random.random() * height,
            'z': random.random() * width,  # For simplicity, use width for depth as well
            'dx': random.random() * 10 - 5,
            'dy': random.random() * 10 - 5,
            'dz': random.random() * 10 - 5,
            'history': [],
        })
    emo_boid_indices = set(random.sample(range(numBoids), int(0.05 * numBoids)))

def distance(boid1, boid2):
    return math.sqrt((boid1['x'] - boid2['x'])**2 + (boid1['y'] - boid2['y'])**2 + (boid1['z'] - boid2['z'])**2)

def nClosestBoids(boid, n):
    sorted_boids = sorted(boids, key=lambda other_boid: distance(boid, other_boid))
    return sorted_boids[1:n+1]

def keepWithinBounds(boid):
    margin = 200
    turnFactor = 1

    if boid['x'] < margin:
        boid['dx'] += turnFactor
    if boid['x'] > width - margin:
        boid['dx'] -= turnFactor
    if boid['y'] < margin:
        boid['dy'] += turnFactor
    if boid['y'] > height - margin:
        boid['dy'] -= turnFactor
    if boid['z'] < margin:
        boid['dz'] += turnFactor
    if boid['z'] > width - margin:
        boid['dz'] -= turnFactor

def flyTowardsCenter(boid):
    centeringFactor = 0.005
    neighbormin = 3
    centerX = 0
    centerY = 0
    centerZ = 0
    numNeighbors = 0

    for otherBoid in boids:
        if distance(boid, otherBoid) < visualRange:
            centerX += otherBoid['x']
            centerY += otherBoid['y']
            centerZ += otherBoid['z']
            numNeighbors += 1

    if numNeighbors <= neighbormin:
        for otherBoid in boids:
            if otherBoid in nClosestBoids(boid, boidneighbor):
                centerX += otherBoid['x']
                centerY += otherBoid['y']
                centerZ += otherBoid['z']
                numNeighbors += 1
            if numNeighbors == neighbormin:
                break

    if numNeighbors:
        centerX /= numNeighbors
        centerY /= numNeighbors
        centerZ /= numNeighbors

        boid['dx'] += (centerX - boid['x']) * centeringFactor
        boid['dy'] += (centerY - boid['y']) * centeringFactor
        boid['dz'] += (centerZ - boid['z']) * centeringFactor

def avoidOthers(boid):
    minDistance = 20
    avoidFactor = 0.05
    moveX = 0
    moveY = 0
    moveZ = 0

    for otherBoid in boids:
        if otherBoid != boid:
            if distance(boid, otherBoid) < minDistance:
                moveX += boid['x'] - otherBoid['x']
                moveY += boid['y'] - otherBoid['y']
                moveZ += boid['z'] - otherBoid['z']

    if boids.index(boid) in emo_boid_indices:
        avoidFactor *= 2  # Emo boids avoid others more aggressively

    boid['dx'] += moveX * avoidFactor
    boid['dy'] += moveY * avoidFactor
    boid['dz'] += moveZ * avoidFactor

def matchVelocity(boid):
    matchingFactor = 0.05
    neighbormin = 3
    avgDX = 0
    avgDY = 0
    avgDZ = 0
    numNeighbors = 0

    for otherBoid in boids:
        if distance(boid, otherBoid) < visualRange:
            avgDX += otherBoid['dx']
            avgDY += otherBoid['dy']
            avgDZ += otherBoid['dz']
            numNeighbors += 1

    if numNeighbors <= neighbormin:
        for otherBoid in boids:
            if otherBoid in nClosestBoids(boid, boidneighbor):
                avgDX += otherBoid['dx']
                avgDY += otherBoid['dy']
                avgDZ += otherBoid['dz']
                numNeighbors += 1
            if numNeighbors == neighbormin:
                break

    if numNeighbors:
        avgDX /= numNeighbors
        avgDY /= numNeighbors
        avgDZ /= numNeighbors

        if boids.index(boid) in emo_boid_indices:
            matchingFactor /= 2  # Emo boids match velocity less

        boid['dx'] += (avgDX - boid['dx']) * matchingFactor
        boid['dy'] += (avgDY - boid['dy']) * matchingFactor
        boid['dz'] += (avgDZ - boid['dz']) * matchingFactor

def limitSpeed(boid):
    speedLimit = 15

    speed = math.sqrt(boid['dx']**2 + boid['dy']**2 + boid['dz']**2)
    if speed > speedLimit:
        boid['dx'] = (boid['dx'] / speed) * speedLimit
        boid['dy'] = (boid['dy'] / speed) * speedLimit
        boid['dz'] = (boid['dz'] / speed) * speedLimit

def drawBoid(boid):
    glBegin(GL_TRIANGLES)
    if boids.index(boid) in emo_boid_indices:
        glColor3f(1, 0, 0)  # Red for emo boids
    else:
        glColor3f(0.33, 0.55, 0.96)  # Blue for normal boids

    # Draw a simple triangle to represent each boid
    glVertex3f(boid['x'], boid['y'], boid['z'])
    glVertex3f(boid['x'] - 5, boid['y'] - 2, boid['z'])
    glVertex3f(boid['x'] - 5, boid['y'] + 2, boid['z'])
    glEnd()

    if DRAW_TRAIL:
        glBegin(GL_LINE_STRIP)
        for point in boid['history']:
            glVertex3f(point[0], point[1], point[2])
        glEnd()

def wind_vector():
    direction = wind_direction
    wind_speed = 0.25  # Adjust the wind speed as needed
    wind_dx = math.cos(direction) * wind_speed
    wind_dy = math.sin(direction) * wind_speed
    wind_dz = math.sin(direction) * wind_speed  # Adding a z-component for wind

    for boid in boids:
        boid['dx'] += wind_dx
        boid['dy'] += wind_dy
        boid['dz'] += wind_dz

# Main animation loop
def animationLoop():
    global boids, frame_count
    frame_count += 1

    for boid in boids:
        flyTowardsCenter(boid)
        avoidOthers(boid)
        matchVelocity(boid)
        limitSpeed(boid)
        keepWithinBounds(boid)

        if frame_count % wind_interval == 0:
            wind_vector()

        boid['x'] += boid['dx']
        boid['y'] += boid['dy']
        boid['z'] += boid['dz']
        boid['history'].append((boid['x'], boid['y'], boid['z']))
        boid['history'] = boid['history'][-50:]

    glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)
    glLoadIdentity()
    gluLookAt(width / 2, height / 2, width * 1.5, width / 2, height / 2, 0, 0, 1, 0)

    for boid in boids:
        drawBoid(boid)

    pygame.display.flip()
    pygame.time.Clock().tick(60)

pygame.init()
pygame.display.set_mode((width, height), DOUBLEBUF | OPENGL)
gluPerspective(45, (width / height), 0.1, 5000.0)
glTranslatef(0, 0, -500)  # Adjust as needed to position the camera
glEnable(GL_DEPTH_TEST)

DRAW_TRAIL = False

initBoids()
while True:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
            sys.exit()
    animationLoop()


SystemExit: 

: 